In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA
from keras.models import Sequential
from keras.layers import Dense, Dropout

**Initially reading input from the dataset, splitting the input into traning and testing sets.**

In [106]:
wdbc = pd.read_csv("wdbc.data")
wdbc = wdbc.iloc[:,:-1]
training,testing = train_test_split(wdbc, test_size=0.3, random_state=42)
# training,testing = train_test_split(wdbc, test_size=0.4, random_state=42)
# training,testing = train_test_split(wdbc, test_size=0.5, random_state=42)
# training,testing = train_test_split(wdbc, test_size=0.2, random_state=42)
# training,testing = train_test_split(wdbc, test_size=0.1, random_state=42)

train_data = training.iloc[:,1:]
test_data = testing.iloc[:,1:]
x_train = train_data.iloc[:,1:]
x_train = MinMaxScaler().fit_transform(x_train)
x_test = test_data.iloc[:,1:]
x_test = MinMaxScaler().fit_transform(x_test)

# Converting the labelled outcomes into numerical data
# Malignant = 0
# Benign = 1
y_train = train_data.iloc[:,:1]
y_train[y_train=='M'] = 0
y_train[y_train=='B'] = 1
print("Training Data :", y_train.shape)

y_test = test_data.iloc[:,:1]
y_test[y_test=='M'] = 0
y_test[y_test=='B'] = 1
print("Testing Data :", y_test.shape)

Training Data : (398, 1)
Testing Data : (171, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

**Looking at the Shape of the training and testing data that we have now :**

In [107]:
print("Training Data :", x_train.shape)
print("Testing Data :", x_test.shape)
print("Training Data :", y_train.shape)
print("Testing Data :", y_test.shape)

Training Data : (398, 30)
Testing Data : (171, 30)
Training Data : (398, 1)
Testing Data : (171, 1)


In [0]:
x = tf.placeholder(tf.float32, [None,30])
y = tf.placeholder(tf.float32, [None, 1])
# weight
weight = tf.Variable(tf.random_normal([30,1], seed=0), name='weight')

# bias
bias = tf.Variable(tf.random_normal([1], seed=0), name='bias')
logits = tf.matmul(x,weight) + bias

layer = tf.nn.sigmoid(logits)
# layer = tf.nn.relu(logits)
# layer = tf.nn.tanh(logits)

exp = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=y)
cost = tf.reduce_mean(exp)

In [0]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
# train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
prediction = tf.cast(layer > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

**Running the Single Layer Perceptron structure, computed accuracies achieved with different number of steps to do a comparitive analysis**

In [110]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # for step in range(5001):
    # for step in range(15001):
    # for step in range(20001):
    for step in range(10001):
        sess.run(train, feed_dict={x: x_train, y: y_train})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: x_train, y: y_train})
            print("Step Number: {:5}\tLoss: {:.3f}\tAccuracy: {:.2%}".format(step, loss, acc))
            
    train_accuracy = sess.run(accuracy, feed_dict={x: x_train, y: y_train})
    test_accuracy,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={x: x_test, y: y_test})
    print("\n")
    print("Model Accuracy Achieved = {:.2%}".format(train_accuracy))
    print("Test Accuracy Achieved = {:.2%}".format(test_accuracy))

Step Number:     0	Loss: 0.848	Accuracy: 39.70%
Step Number:  1000	Loss: 0.238	Accuracy: 91.21%
Step Number:  2000	Loss: 0.180	Accuracy: 94.72%
Step Number:  3000	Loss: 0.154	Accuracy: 96.23%
Step Number:  4000	Loss: 0.138	Accuracy: 96.98%
Step Number:  5000	Loss: 0.128	Accuracy: 97.49%
Step Number:  6000	Loss: 0.120	Accuracy: 97.74%
Step Number:  7000	Loss: 0.114	Accuracy: 97.99%
Step Number:  8000	Loss: 0.110	Accuracy: 98.24%
Step Number:  9000	Loss: 0.106	Accuracy: 98.24%
Step Number: 10000	Loss: 0.102	Accuracy: 98.24%


Model Accuracy Achieved = 98.24%
Test Accuracy Achieved = 94.74%


**The next methods tried is using the same Single Layer Perceptron structure, but applying Principal Component Analysis first. Here various number of components were tested out.**

In [0]:
# comp is the number of PCA Components [passed]
comp = 15
pca = sklearnPCA(n_components = comp)
pca_train_x = pca.fit_transform(x_train)
pca_test_x = pca.fit_transform(x_test)
x = tf.placeholder(tf.float32, [None,comp])
y = tf.placeholder(tf.float32, [None, 1])

weight = tf.Variable(tf.random_normal([comp,1], seed=0), name='weight')
bias = tf.Variable(tf.random_normal([1], seed=0), name='bias')
logits = tf.matmul(x,weight) + bias
layer = tf.nn.sigmoid(logits)
exp = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=y)
cost = tf.reduce_mean(exp)
train = tf.train.GradientDescentOptimizer(learning_rate=0.2).minimize(cost)

prediction = tf.cast(layer > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    

**Running the PCA SLP Model:**

In [112]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={x: pca_train_x, y: y_train})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: pca_train_x, y: y_train})
            print("Step Number: {:5}\tLoss: {:.3f}\tAccuracy: {:.2%}".format(step, loss, acc))

    train_accuracy = sess.run(accuracy, feed_dict={x: pca_train_x, y: y_train})
    test_accuracy,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={x: pca_test_x, y: y_test})
    
print("\n")
print("Model Accuracy Achieved = {:.2%}".format(train_accuracy))
print("Test Accuracy Achieved = {:.2%}".format(test_accuracy))
        

Step Number:     0	Loss: 0.700	Accuracy: 54.52%
Step Number:  1000	Loss: 0.141	Accuracy: 95.98%
Step Number:  2000	Loss: 0.116	Accuracy: 96.98%
Step Number:  3000	Loss: 0.105	Accuracy: 97.49%
Step Number:  4000	Loss: 0.098	Accuracy: 97.74%
Step Number:  5000	Loss: 0.094	Accuracy: 97.74%
Step Number:  6000	Loss: 0.090	Accuracy: 97.74%
Step Number:  7000	Loss: 0.088	Accuracy: 97.74%
Step Number:  8000	Loss: 0.085	Accuracy: 97.99%
Step Number:  9000	Loss: 0.084	Accuracy: 97.99%
Step Number: 10000	Loss: 0.082	Accuracy: 97.99%


Model Accuracy Achieved = 97.99%
Test Accuracy Achieved = 96.49%


**Running a Multi Layer Perceptron Structure after Implementing PCA**

In [0]:
comp = 5
pca = sklearnPCA(n_components=comp)
pca_mlp_train_x = pca.fit_transform(x_train)

pca_mlp_test_x = pca.fit_transform(x_test)

x = tf.placeholder(tf.float32, [None,comp])
y = tf.placeholder(tf.float32, [None, 1])

# input layer
weight1 = tf.Variable(tf.random_normal([comp,64], seed=0), name='weight1')
bias1 = tf.Variable(tf.random_normal([64], seed=0), name='bias1')
layer1 = tf.nn.sigmoid(tf.matmul(x,weight1) + bias1)

weight2 = tf.Variable(tf.random_normal([64,128], seed=0), name='weight2')
bias2 = tf.Variable(tf.random_normal([128], seed=0), name='bias2')
layer2 = tf.nn.sigmoid(tf.matmul(layer1,weight2) + bias2)
weight3 = tf.Variable(tf.random_normal([128,128], seed=0), name='weight3')
bias3 = tf.Variable(tf.random_normal([128], seed=0), name='bias3')
layer3 = tf.nn.sigmoid(tf.matmul(layer2,weight3) + bias3)

# output layer
weight4 = tf.Variable(tf.random_normal([128,1], seed=0), name='weight4')
bias4 = tf.Variable(tf.random_normal([1], seed=0), name='bias4')
logits = tf.matmul(layer3,weight4) + bias4
output_layer = tf.nn.sigmoid(logits)
        

In [0]:
exp = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=y)
cost = tf.reduce_mean(exp)
train = tf.train.GradientDescentOptimizer(learning_rate=0.005).minimize(cost)

prediction = tf.cast(output_layer > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

**Running the PCA MLP Model:**

In [115]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={x: pca_mlp_train_x, y: y_train})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: pca_mlp_train_x, y: y_train})
            print("Step Number: {:5}\tLoss: {:.3f}\tAccuracy: {:.2%}".format(step, loss, acc))

    train_accuracy = sess.run(accuracy, feed_dict={x: pca_mlp_train_x, y: y_train})
    test_accuracy,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={x: pca_mlp_test_x, y: y_test})

print("\n")
print("Model Accuracy Achieved = {:.2%}".format(train_accuracy))
print("Test Accuracy Achieved = {:.2%}".format(test_accuracy))

Step Number:     0	Loss: 3.107	Accuracy: 62.56%
Step Number:  1000	Loss: 0.146	Accuracy: 96.23%
Step Number:  2000	Loss: 0.113	Accuracy: 96.48%
Step Number:  3000	Loss: 0.099	Accuracy: 96.98%
Step Number:  4000	Loss: 0.091	Accuracy: 97.24%
Step Number:  5000	Loss: 0.085	Accuracy: 97.24%
Step Number:  6000	Loss: 0.081	Accuracy: 97.24%
Step Number:  7000	Loss: 0.078	Accuracy: 97.24%
Step Number:  8000	Loss: 0.076	Accuracy: 97.49%
Step Number:  9000	Loss: 0.074	Accuracy: 97.49%
Step Number: 10000	Loss: 0.072	Accuracy: 97.74%


Model Accuracy Achieved = 97.74%
Test Accuracy Achieved = 97.08%


**Alternative MLP Classification using Keras (Without PCA) (Implementation only for testing)**



In [116]:
model = Sequential()
model.add(Dense(64, input_dim=30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/100
398/398 [==============================] - 0s 230us/step - loss: 0.6906 - accuracy: 0.6281
Epoch 2/100
398/398 [==============================] - 0s 35us/step - loss: 0.7167 - accuracy: 0.5628
Epoch 3/100
398/398 [==============================] - 0s 31us/step - loss: 0.6483 - accuracy: 0.6457
Epoch 4/100
398/398 [==============================] - 0s 31us/step - loss: 0.7207 - accuracy: 0.5980
Epoch 5/100
398/398 [==============================] - 0s 33us/step - loss: 0.6597 - accuracy: 0.6080
Epoch 6/100
398/398 [==============================] - 0s 34us/step - loss: 0.6713 - accuracy: 0.6080
Epoch 7/100
398/398 [==============================] - 0s 29us/step - loss: 0.6072 - accuracy: 0.6558
Epoch 8/100
398/398 [==============================] - 0s 40us/step - loss: 0.6202 - accuracy: 0.6407
Epoch 9/100
398/398 [==============================] - 0s 36us/step - loss: 0.5716 - accuracy: 0.7312
Epoch 10/100
398/398 [==============================] - 0s 34us/step - loss: 0.58